In [1]:
!git clone https://github.com/pkulwj1994/latent-diffusion.git
!cp -a /content/latent-diffusion/. /content/.

Cloning into 'latent-diffusion'...
remote: Enumerating objects: 282, done.
remote: Total 282 (delta 0), reused 0 (delta 0), pack-reused 282
Receiving objects: 100% (282/282), 28.39 MiB | 8.83 MiB/s, done.
Resolving deltas: 100% (104/104), done.


In [ ]:
!mkdir -p models/ldm/text2img-large/
!wget -O models/ldm/text2img-large/model.ckpt https://ommer-lab.com/files/latent-diffusion/nitro/txt2img-f8-large/model.ckpt

--2022-05-22 12:17:05--  https://ommer-lab.com/files/latent-diffusion/nitro/txt2img-f8-large/model.ckpt
Resolving ommer-lab.com (ommer-lab.com)... 141.84.41.65
Connecting to ommer-lab.com (ommer-lab.com)|141.84.41.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6152314307 (5.7G)
Saving to: ‘models/ldm/text2img-large/model.ckpt’

models/ldm/text2img  73%[=============>      ]   4.22G   130MB/s    in 34s     

2022-05-22 12:17:39 (126 MB/s) - Connection closed at byte 4527210496. Retrying.

--2022-05-22 12:17:40--  (try: 2)  https://ommer-lab.com/files/latent-diffusion/nitro/txt2img-f8-large/model.ckpt
Connecting to ommer-lab.com (ommer-lab.com)|141.84.41.65|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 6152314307 (5.7G), 1625103811 (1.5G) remaining
Saving to: ‘models/ldm/text2img-large/model.ckpt’

ext2img-large/model  82%[++++++++++++++=>    ]   4.75G   128MB/s    eta 9s     

In [10]:
!pip install omegaconf
!pip install einops
!pip uninstall ldm
!pip install pytorch_lightning
!pip install fvcore==0.1.5.post20210924
!pip install .
!pip install transformers==4.3.1
# !pip install ldm
# !export PYTHONPATH=$PYTHONPATH:$(pwd)

# !git clone https://github.com/CompVis/taming-transformers.git
# %cd taming-transformers
# !pip install .
# %cd ..
!pip install taming-transformers

!git clone https://github.com/openai/CLIP.git
%cd CLIP
!pip install .
%cd ..

!git clone https://github.com/CompVis/taming-transformers.git
%cd taming-transformers
!pip install .
%cd ..

Processing /content
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for latent-diffusion: filename=latent_diffusion-0.0.1-py3-none-any.whl size=1950 sha256=ac3043b8782a6071de87d63eb69a770e5387b844f7f2ca1c16a45a24355f9051
  Stored in directory: /tmp/pip-ephem-wheel-cache-012ij4ve/wheels/63/0e/3b/c99faf0b2fc9bcbb6d311ff482288076f10040d8aef364be10
Successfully built latent-diffusion
  Attempting uninstall: latent-diffusion
    Found existing installation: latent-diffusion 0.0.1
    Uninstalling latent-diffusion-0.0.1:
      Successfully uninstalled latent-diffusion-0.0.1
     |████████████████████████████████| 1.8 MB 15.0 MB/s 
   

In [8]:
!cp -a taming-transformers/taming taming

In [13]:
!cp scripts/txt2img.py txt2img.py
!python txt2img.py --prompt "a virus monster is playing guitar, oil on canvas" --ddim_eta 0.0 --n_samples 4 --n_iter 4 --scale 5.0  --ddim_steps 50


Loading model from models/ldm/text2img-large/model.ckpt
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 872.30 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
^C


In [1]:
import argparse, os, sys, glob
import torch
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm, trange
from einops import rearrange
from torchvision.utils import make_grid

from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler


def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cuda")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.cuda()
    model.eval()
    return model

args = ['--prompt', 'a virus monster is playing guitar, oil on canvas', '--ddim_eta', '0.0', '--n_samples', '4', '--n_iter', '4', '--scale', '5.0',  '--ddim_steps', '50']


parser = argparse.ArgumentParser()

parser.add_argument(
    "--prompt",
    type=str,
    nargs="?",
    default="a painting of a virus monster playing guitar",
    help="the prompt to render"
)

parser.add_argument(
    "--outdir",
    type=str,
    nargs="?",
    help="dir to write results to",
    default="outputs/txt2img-samples"
)
parser.add_argument(
    "--ddim_steps",
    type=int,
    default=200,
    help="number of ddim sampling steps",
)

parser.add_argument(
    "--plms",
    action='store_true',
    help="use plms sampling",
)

parser.add_argument(
    "--ddim_eta",
    type=float,
    default=0.0,
    help="ddim eta (eta=0.0 corresponds to deterministic sampling",
)
parser.add_argument(
    "--n_iter",
    type=int,
    default=1,
    help="sample this often",
)

parser.add_argument(
    "--H",
    type=int,
    default=256,
    help="image height, in pixel space",
)

parser.add_argument(
    "--W",
    type=int,
    default=256,
    help="image width, in pixel space",
)

parser.add_argument(
    "--n_samples",
    type=int,
    default=4,
    help="how many samples to produce for the given prompt",
)

parser.add_argument(
    "--scale",
    type=float,
    default=5.0,
    help="unconditional guidance scale: eps = eps(x, empty) + scale * (eps(x, cond) - eps(x, empty))",
)
opt = parser.parse_args(args)


config = OmegaConf.load("configs/latent-diffusion/txt2img-1p4B-eval.yaml")  # TODO: Optionally download from same location as ckpt and chnage this logic
model = load_model_from_config(config, "models/ldm/text2img-large/model.ckpt")  # TODO: check path

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)

if opt.plms:
    sampler = PLMSSampler(model)
else:
    sampler = DDIMSampler(model)

os.makedirs(opt.outdir, exist_ok=True)
outpath = opt.outdir

prompt = opt.prompt


sample_path = os.path.join(outpath, "samples")
os.makedirs(sample_path, exist_ok=True)
base_count = len(os.listdir(sample_path))

all_samples=list()
with torch.no_grad():
    with model.ema_scope():
        uc = None
        if opt.scale != 1.0:
            uc = model.get_learned_conditioning(opt.n_samples * [""])
        for n in trange(opt.n_iter, desc="Sampling"):
            c = model.get_learned_conditioning(opt.n_samples * [prompt])
            shape = [4, opt.H//8, opt.W//8]
            samples_ddim, _ = sampler.sample(S=opt.ddim_steps,
                                              conditioning=c,
                                              batch_size=opt.n_samples,
                                              shape=shape,
                                              verbose=False,
                                              unconditional_guidance_scale=opt.scale,
                                              unconditional_conditioning=uc,
                                              eta=opt.ddim_eta)

            x_samples_ddim = model.decode_first_stage(samples_ddim)
            x_samples_ddim = torch.clamp((x_samples_ddim+1.0)/2.0, min=0.0, max=1.0)

            for x_sample in x_samples_ddim:
                x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                Image.fromarray(x_sample.astype(np.uint8)).save(os.path.join(sample_path, f"{base_count:04}.png"))
                base_count += 1
            all_samples.append(x_samples_ddim)


# additionally, save as grid
grid = torch.stack(all_samples, 0)
grid = rearrange(grid, 'n b c h w -> (n b) c h w')
grid = make_grid(grid, nrow=opt.n_samples)

# to image
grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
Image.fromarray(grid.astype(np.uint8)).save(os.path.join(outpath, f'{prompt.replace(" ", "-")}.png'))

print(f"Your samples are ready and waiting four you here: \n{outpath} \nEnjoy.")


Loading model from models/ldm/text2img-large/model.ckpt
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 872.30 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Sampling:   0%|          | 0/4 [00:00<?, ?it/s]

Data shape for DDIM sampling is (4, 4, 32, 32), eta 0.0
Running DDIM Sampling with 50 timesteps



Sampling:  25%|██▌       | 1/4 [00:08<00:26,  8.79s/it]

Data shape for DDIM sampling is (4, 4, 32, 32), eta 0.0
Running DDIM Sampling with 50 timesteps


...

Data shape for DDIM sampling is (4, 4, 32, 32), eta 0.0
Running DDIM Sampling with 50 timesteps



Sampling:  75%|███████▌  | 3/4 [00:26<00:08,  8.64s/it]

Data shape for DDIM sampling is (4, 4, 32, 32), eta 0.0
Running DDIM Sampling with 50 timesteps



Sampling: 100%|██████████| 4/4 [00:34<00:00,  8.59s/it]


Your samples are ready and waiting four you here: 
outputs/txt2img-samples 
Enjoy.
